# Northern Ireland UBI Feasibility Study

*Research supported by the UBI Lab Northern Ireland.*

* Amount: £200, 300, 400 per month
* Half shares for children and pensioners
* UBI included in neither means tests nor income tax
* Personal tax allowance at £2k
* Other rates from there, prioritizing higher 

## Tax and benefit policy in Northern Ireland


## Costing the UBI

* Cost of UBI up to £400 per month
* Start with amount from personal allowance
* Show equivalence between each rate increase


In [ ]:
from policyengine.countries import UK

uk = UK()

baseline, reformed = uk._get_microsimulations(ubini, refresh_baseline=True)


In [24]:
from policyengine.countries import UK
from openfisca_uk.model_api import *
from openfisca_uk.api import *
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

uk = UK()


CHILD_PENSIONER_SHARE = 0.5


def make_ubi(adult_ubi, personal_allowance, basic_rate, higher_rate, additional_rate):

    def change_parameters(parameters):
        PERIOD = "year:2022:10"
        # UBI.
        if adult_ubi is not None:
            parameters.contrib.ubi_center.basic_income.amount.by_age.working_age.update(
                period=periods.period(PERIOD), value=adult_ubi
            )
            child_senior_ubi = adult_ubi * CHILD_PENSIONER_SHARE
            parameters.contrib.ubi_center.basic_income.amount.by_age.child.update(
                period=periods.period(PERIOD), value=child_senior_ubi
            )
            parameters.contrib.ubi_center.basic_income.amount.by_age.senior.update(
                period=periods.period(PERIOD), value=child_senior_ubi
            )
        # Tax parameters.
        if personal_allowance is not None:
            parameters.tax.income_tax.allowances.personal_allowance.amount.update(
                period=periods.period(PERIOD), value=personal_allowance
            )
        if basic_rate is not None:
            parameters.tax.income_tax.rates.uk.brackets[0].rate.update(
                period=periods.period(PERIOD), value=basic_rate
            )
        if higher_rate is not None:
            parameters.tax.income_tax.rates.uk.brackets[1].rate.update(
                period=periods.period(PERIOD), value=higher_rate
            )
        if additional_rate is not None:
            parameters.tax.income_tax.rates.uk.brackets[2].rate.update(
                period=periods.period(PERIOD), value=additional_rate
            )
        parameters.reforms.country_specific.update(
            period=periods.period(PERIOD), value=True
        )
        return parameters


    class ubini(Reform):
        def apply(self):
            self.modify_parameters(change_parameters)

    return ubini


In [15]:
l = []
for ubi in range(0, 401, 50):
    res = uk.population_reform(make_ubi(ubi, None, None, None, None))
    l.append(pd.Series(dict(ubi=ubi, cost=res["net_cost_numeric"])))
ubi_cost = pd.DataFrame(l)

In [18]:
fig = px.line(ubi_cost, "ubi", "cost")
format_fig(fig)

In [12]:
res = uk.population_reform(make_ubi(200, 0, 0.2, 0.4, 0.45))
res["net_cost"]
res.keys()

'£450.6bn'

In [13]:
res.keys()

dict_keys(['net_cost', 'net_cost_numeric', 'poverty_change', 'winner_share', 'loser_share', 'gini_change', 'rel_income_decile_chart', 'avg_income_decile_chart', 'rel_wealth_decile_chart', 'avg_wealth_decile_chart', 'poverty_chart', 'deep_poverty_chart', 'waterfall_chart', 'intra_income_decile_chart', 'intra_wealth_decile_chart', 'inequality_chart'])

In [25]:
baseline, reformed = uk._get_microsimulations(ubini)

NI = "NORTHERN_IRELAND"


def filter_weights(sim):
    sim.set_input(
        "household_weight",
        sim.year,
        np.where(sim.calc("country") == NI, sim.calc("household_weight"), 0),
    )
    sim.set_input(
        "person_weight",
        sim.year,
        np.where(
            sim.calc("country", map_to="person") == NI,
            sim.calc("person_weight"),
            0,
        ),
    )


filter_weights(baseline)
filter_weights(reformed)


In [26]:
res = uk.population_reform(ubini)
res["net_cost"]


'£1.09tr'

## Impacts of budget-neutral UBI plans

* Poverty
* Inequality
